In [37]:
%logstart -o -t

import sys

import numpy as np
#in case we need to repeat experiment
np.random.seed(255)

import pandas as pd
pd.options.display.max_rows = 22

import matplotlib.pyplot as plt
plt.style.use('classic')

import seaborn as sns
sns.set()

#sklearn imports
from sklearn.cluster import KMeans
#from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV


#Use print instead of display when run as python script
pyscript = True

#Classifier verborsity where supported
verbose_level=1

#Sample Size (5963272)
#sampleN = 3000000
SampleN = 5963000

import time

#Enable Optimization Algorithms
enable_grid_search = False
svm_c = 1
svm_gamma = 1
enable_rf_features = True
feature_all = False
defaultFeatures = ['P_AGE', 'V_YEAR', 'C_HOUR', 'C_YEAR', 'C_MNTH', 'C_CONF', 'C_WDAY', 'C_VEHS', 'P_USER', 'P_SEX']
enable_kmeans = True
enable_logistic_regression_l1 = True

# Enable Algorithms
enable_logistic_regression = True
enable_decision_tree = True
enable_svm = False
enable_knn = True
enable_mlp = True
enable_rf = True

#Multiclass classification, binary if falase
multiclass = False

Activating auto-logging. Current session state plus future input saved.
Filename       : ipython_log.py
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


In [38]:
print("Sample size: {}".format(sampleN))

if multiclass:
    print("Multi-Class Classification: Enabled")
else:
    print("Multi-Class Classification: Disabled")

if enable_grid_search:
    print("Grid Search: Enabled")
else:
    print("Grid Search: Disabled")

if enable_rf_features:
    print("Feature Selection by RandomForest: Enabled")
else:
    print("Feature Selection by RandomForest: Disabled")
    print("Default Feature: {}".format(defaultFeatures))

if feature_all:
    print("All Features: Enabled")
else:
    print("All Features: Disabled")
    
if enable_kmeans:
    print("K-means: Enabled")
else:
    print("K-means: Disabled")

if enable_logistic_regression:
    print("Logistic Regression: Enabled")
else:
    print("Logistic Regression: Disabled")
    
if enable_decision_tree:
    print("Decision Tree: Enabled")
else:
    print("Decision Tree: Disabled")
    
if enable_svm:
    print("Support Vector Machines: Enabled")
else:
    print("Support Vector Machines: Disabled")

if  enable_knn:
    print("KNN: Enabled")
else:
    print("KNN: Disabled")
    
if enable_mlp:
    print("MLP: Enabled")
else:
    print("MLP: Disabled")
    
if enable_rf:
    print("Random Forest: Enabeld")
else:
    print("Random Forest: Disabled")

Sample size: 3000000
Multi-Class Classification: Disabled
Grid Search: Disabled
Feature Selection by RandomForest: Enabled
All Features: Disabled
K-means: Enabled
Logistic Regression: Enabled
Decision Tree: Enabled
Support Vector Machines: Disabled
KNN: Enabled
MLP: Enabled
Random Forest: Enabeld


In [39]:
#sys.stdout=open("Initial_Binary_Model2.out", "w")

In [40]:
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ))

Mon Nov 12 09:03:33 2018


In [41]:
#df = pd.read_csv('data01_simple.csv', engine = 'python')
df = pd.read_csv('data01_clean.csv', engine = 'python')
print(df.head(2))

   C_YEAR  C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  \
0    1999       1       1      20       2      34       8       1       5   
1    1999       1       1      20       2      34       2       1       5   

   C_RALN   ...    V_ID  V_TYPE  V_YEAR  P_ID  P_SEX  P_AGE  P_PSN  P_SAFE  \
0       3   ...       1       6    1990     1      1     41     11       2   
1       3   ...       2       1    1987     1      1     19     11       2   

   P_USER  P_ISEV  
0       1       1  
1       1       1  

[2 rows x 21 columns]


In [42]:
print(df.isnull().sum().sum())

0


In [43]:
print(df[df.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())

0


In [44]:
df_cat = df.astype('category').copy()

In [45]:
df_int = df.astype('int').copy()

In [46]:
totalRows = df_cat.index.size
print("Number of Rows in the dataset: {}".format(totalRows))

Number of Rows in the dataset: 5963272


In [47]:
print(df_cat.columns)

Index(['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG',
       'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_ID', 'V_TYPE', 'V_YEAR',
       'P_ID', 'P_SEX', 'P_AGE', 'P_PSN', 'P_SAFE', 'P_USER', 'P_ISEV'],
      dtype='object')


In [48]:
#One-Hot-Encoding of categorical


## Convert Class Variable to Binary if multi class disabled

In [49]:
## Convert Class Variable to Binary
### Merge Injury and Fatality as a single class
### we will compare the results.
if multiclass:
    #Undersample the majority for the 3 class evaluation
    
    df_class = df_cat.copy()
    
    # subset fatal class
    is_fatal =  df_class['P_ISEV']==3
    is_fatal_count = is_fatal.sum()
    print("Number of Fatal: {}".format(is_fatal_count))
    df_class_fatal = df_class[is_fatal]
    print(df_class_fatal.head(2))
    
    # subset injury class
    is_injury =  df_class['P_ISEV']==2
    is_injury_count = is_injury.sum()
    print("Number of Injury: {}".format(is_injury_count))
    df_class_injury = df_class[is_injury]
    print(df_class_injury.head(2))
    
    # subset non_injury class
    is_safe =  df_class['P_ISEV']==1
    is_safe_count = is_safe.sum()
    print("Number of Non-Injury: {}".format(is_safe_count))
    df_class_safe = df_class[is_safe]
    print(df_class_safe.head(2))
    
    # get the size of fatal datafram
    min_size = df_class_fatal.index.size
    print("Size of Fatal Subset: {}".format(min_size))
    
    # get size of injury
    print("Size of injury Subset: {}".format(df_class_injury.index.size))
    
    # size of non-fatal
    print("Size of non-fatal Subset: {}".format(df_class_safe.index.size))
    
    # randomly sample n number of injury and no injury and append to fatal
    df_class_injury_select = df_class_injury.sample(n=min_size)
    print("Shape of injury sampled dataframe: {}".format(df_class_injury_select.shape))
    df_class_safe_select = df_class_safe.sample(n=min_size)
    print("Shape of nom-injury sampled dataframe: {}".format(df_class_safe_select.shape))
    
    #concat the three dataframes
    df_underSample = pd.concat([df_class_fatal, df_class_injury_select, df_class_safe_select])
    print(df_underSample.shape)
    
    #TBD
    if sampleN < df_underSample.index.size:
        df_sample = df_underSample.sample(n=sampleN)
    else:
        df_sample = df_underSample.sample(n=df_underSample.index.size)
    
    #perform the conversion in two steps to avoid any unwanted side effects
    df_sample['P_ISEV'] = df_sample['P_ISEV'].map({1: 'safe', 2: 'injury', 3:'fatal'})
    df_sample['P_ISEV'] = df_sample['P_ISEV'].map({'safe': '0', 'injury': '1', 'fatal':'2'})
    print((df_sample['P_ISEV']=='0').sum())
    print((df_sample['P_ISEV']=='1').sum())
    print((df_sample['P_ISEV']=='2').sum())
    print(df_sample['P_ISEV'].unique())
else:
    df_class = df_cat.copy()

    #perform the conversion in two steps to avoid any unwanted side effects
    df_class['P_ISEV'] = df_class['P_ISEV'].map({1: 'safe', 2: 'injury', 3:'fatal'})
    df_class['P_ISEV'] = df_class['P_ISEV'].map({'safe': '0', 'injury': '1', 'fatal':'1'})
    print((df_class['P_ISEV']=='0').sum())
    print((df_class['P_ISEV']=='1').sum())
    print(df_class['P_ISEV'].unique())
    
    df_sample = df_class.sample(n=sampleN)

print("Size of dataframe for modeling: {}".format(df_sample.index.size))

2568097
3395175
['0' '1']
Size of dataframe for modeling: 3000000


In [50]:
print(df_sample[df_sample.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())

0


## Split Training and Testing for Binary class

In [51]:
#Split between data and class
Y = df_sample[df_sample.columns[-1]]
X = df_sample[df_sample.columns[0:df_sample.columns.size -1]]

#### Split Test(70%) and Train (30%) for Bianry class 

In [52]:
#sprint into train and test 70/30
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [53]:
#print(Xbinary_train, Xbinary_test, Ybinary_train, Ybinary_test)

## Write cleaned data to file for future use.

In [54]:
#lets write the datafile for future use
df_sample.to_csv('cleansimple_binary.csv', encoding='utf-8', index=False)

## Clustering based on K-Means Clustering

In [55]:
if enable_kmeans:
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    print("K-Means Clustering: Start")
    kmeans = KMeans(n_clusters=3, init='random', n_init=10, max_iter=100, tol=1e-04, verbose= verbose_level)
    print("K-Means Clustering: Build")
    ykm = kmeans.fit(X)
    
    if pyscript:
        print(ykm.cluster_centers_)
        print(ykm.labels_)
    else:
        display(ykm.cluster_centers_)
        display(ykm.labels_)
    
    print("K-Means Clustering: End")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))


Mon Nov 12 09:08:08 2018
K-Means Clustering: Start
K-Means Clustering: Build
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 3445536841.2893405
start iteration
done sorting
end inner loop
Iteration 1, inertia 3298901419.5318637
start iteration
done sorting
end inner loop
Iteration 2, inertia 3204974298.6932626
start iteration
done sorting
end inner loop
Iteration 3, inertia 3156972202.6493087
start iteration
done sorting
end inner loop
Iteration 4, inertia 3094043887.5391054
start iteration
done sorting
end inner loop
Iteration 5, inertia 2963292622.0892034
start iteration
done sorting
end inner loop
Iteration 6, inertia 2593071052.3717756
start iteration
done sorting
end inner loop
Iteration 7, inertia 1684200430.4752574
start iteration
done sorting
end inner loop
Iteration 8, inertia 1676109291.1682737
start iteration
done sorting
end inner loop
Iteration 9, inertia 1671814975.452863
start iteration
done sorting
end inner loop
Iteration 10, in

Iteration 6, inertia 1677768591.5363572
start iteration
done sorting
end inner loop
Iteration 7, inertia 1672718550.6315958
start iteration
done sorting
end inner loop
Iteration 8, inertia 1657227094.5166745
start iteration
done sorting
end inner loop
Iteration 9, inertia 1644526735.3258054
start iteration
done sorting
end inner loop
Iteration 10, inertia 1644414618.6451957
start iteration
done sorting
end inner loop
Iteration 11, inertia 1644382455.6387856
start iteration
done sorting
end inner loop
Iteration 12, inertia 1644372665.505125
center shift 7.040363e-02 within tolerance 6.842688e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 3304531553.624642
start iteration
done sorting
end inner loop
Iteration 1, inertia 3250855954.949483
start iteration
done sorting
end inner loop
Iteration 2, inertia 3179561755.432772
start iteration
done sorting
end inner loop
Iteration 3, inertia 3069181260.093077
start iteration
done sorting
end inner loo

## Feature selection using Random Forest

### Feature Selection 

### Get the inportant features from random forest

In [56]:
if enable_rf_features:
    print("Random Forest Feature Selection: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    forest = RandomForestClassifier(n_estimators=250, random_state=0, n_jobs=-1, verbose=verbose_level)
    print("Random Forest Feature Selection: Fit Start")
    forest.fit(X, Y)
    print("Random Forest Feature Selection: Fit")

    importFeatures = forest.feature_importances_
    print("Random Forest Feature Selection: Feature Importance")
    print(importFeatures)
    
    indices = np.argsort(importFeatures)[::-1]
    print(indices)
    featureLabel = X.columns[0:]
    print(featureLabel)
    rankedFeature = []
    for f in range(X.shape[1]):
        rankedFeature.append(featureLabel[indices[f]])
        print("%2d) %-*s %f" % (f+1, 30,  featureLabel[indices[f]], importFeatures[indices[f]]))
    print(rankedFeature)

Random Forest Feature Selection: Start
Mon Nov 12 09:10:08 2018
Random Forest Feature Selection: Fit Start


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 12.4min finished


Random Forest Feature Selection: Fit
Random Forest Feature Selection: Feature Importance
[0.10004576 0.0874758  0.06620822 0.10393389 0.03118463 0.06291441
 0.02563893 0.02337054 0.02388434 0.02666642 0.02970882 0.0284344
 0.02492271 0.12025017 0.01364572 0.02765456 0.14062123 0.02332973
 0.01305671 0.027053  ]
[16 13  3  0  1  2  5  4 10 11 15 19  9  6 12  8  7 17 14 18]
Index(['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG',
       'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_ID', 'V_TYPE', 'V_YEAR',
       'P_ID', 'P_SEX', 'P_AGE', 'P_PSN', 'P_SAFE', 'P_USER'],
      dtype='object')
 1) P_AGE                          0.140621
 2) V_YEAR                         0.120250
 3) C_HOUR                         0.103934
 4) C_YEAR                         0.100046
 5) C_MNTH                         0.087476
 6) C_WDAY                         0.066208
 7) C_CONF                         0.062914
 8) C_VEHS                         0.031185
 9) C_TRAF                         0

### Reduce the number of features

In [57]:
#select features that contribute more than 0.05
if enable_rf_features:
    X_Selected = X[rankedFeature[0:10]]
else:
    if feature_all:
        X_Selected = X
    else:
        X_Selected = X[defaultFeatures]

if pyscript:
    print(X_Selected)
else:
    display(X_Selected)

t_end =  time.time()
print(time.asctime( time.localtime(t_end) ))

print("Random Forest Feature Selection: End")


        P_AGE V_YEAR C_HOUR C_YEAR C_MNTH C_WDAY C_CONF C_VEHS C_TRAF V_ID
3564056    25   1993      8   2008     10      5     21      2     18    2
1569277     6   2002      8   2003      1      5     21      2      1    1
3834571    28   1989     19   2009      9      3      6      1     18    1
824407     16   1985     23   2001      2      5      6      1     18    1
3515329    12   2006     20   2008      8      7     35      2      1    1
3441590    31   1998     13   2008      6      2     35      6      1    5
5243339    59   2018     13   2014      6      7      4      1     18    1
4698727    55   2000      7   2012      8      2     36      2     18    2
484740     64   1991     10   2000      4      4     21      2     18    2
2112369    67   1993     11   2004      7      6      3      1     18    1
4270477    68   2018      9   2011      2      4     36      2      1    2
...       ...    ...    ...    ...    ...    ...    ...    ...    ...  ...
538236     40   1984     

In [58]:
print("Split Test and Train based on Selected Features")
#sprint into train and test 70/30
X_train, X_test, Y_train, Y_test = train_test_split(X_Selected, Y, test_size=0.3, random_state=0)

Split Test and Train based on Selected Features


### SVM GridSearch for Optimal Parms

In [59]:
#This operation is computationaly expensive.
if enable_grid_search:
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    param_grid = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}
    grid = GridSearchCV(SVC(), param_grid, verbose=verbose_level)
    grid.fit(X_train, Y_train)
    print(grid.best_params_)
    svm_c = grid.best_params_.get('C')
    svm_gamma = grid.best_params_.get('gamma')
    print(grid.best_estimator_)
    grid_predictions = grid.predict(X_test)
    cfn_matrix_grid = confusion_matrix(Y_test, grid_predictions)
    print(cfn_matrix_grid)
    print(classification_report(Y_test,grid_predictions))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))

## Logistic Regression Model

In [60]:
if enable_logistic_regression:
    print("Logistic Regression: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    lr = LogisticRegression(C=1, random_state=0, verbose=verbose_level)
    print("Logistic Regression: Fit")
    lr.fit(X_train, Y_train)
    print("Logistic Regression: Predict")
    y_pred = lr.predict(X_test)

    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    # print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Intercept")
    print(lr.intercept_)

    # print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Coefficients")
    print(lr.coef_)

    print("Logistic Regression: Confusion Matrix")
    cnf_matrix_lg = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg)
    
    print("Logistic Regression: Classification Report")
    print(classification_report(Y_test, y_pred))

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Logistic Regression: End")

Logistic Regression: Start
Mon Nov 12 09:22:37 2018
Logistic Regression: Fit
[LibLinear]Logistic Regression: Predict
Accuracy of logistic regression classifier on train set: 0.59
Accuracy of logistic regression classifier on test set: 0.59
Logistic Regression: Intercept
[0.00013692]
Logistic Regression: Coefficients
[[ 0.0039192   0.00283072 -0.01248261 -0.00243132 -0.0011258  -0.00060962
  -0.00440272 -0.32099888  0.01747727  0.03448218]]
Logistic Regression: Confusion Matrix
[[108810 278531]
 [ 89995 422664]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.55      0.28      0.37    387341
          1       0.60      0.82      0.70    512659

avg / total       0.58      0.59      0.56    900000

Mon Nov 12 09:23:17 2018
Logistic Regression: End


### Logistic Regression with L1 Regularization

In [ ]:
if enable_logistic_regression_l1:
    # with L1 regularization
    print("Logistic Regression with L1 Regularization: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    lr = LogisticRegression(penalty='l1', C=1, random_state=0, verbose=verbose_level)
    print("Logistic Regression with L1 Regularization: Fit")
    lr.fit(X_train, Y_train)
    
    print("Logistic Regression with L1 Regularization: Predict")
    y_pred = lr.predict(X_test)
    
    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    print("Logistic Regression with L1 Regularization: Confusion Matrix")
    cnf_matrix_lg_l1 = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg_l1)
    
    print(classification_report(Y_test,y_pred))
    print("Logistic Regression with L1 Regularization: Classification Report")

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Logistic Regression with L1 Regularization: End")

Logistic Regression with L1 Regularization: Start
Mon Nov 12 09:23:17 2018
Logistic Regression with L1 Regularization: Fit
[LibLinear]

### Decision Tree

In [ ]:
if enable_decision_tree:
    print("Decision Tree: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    tree = DecisionTreeClassifier(criterion='entropy',max_depth=50, random_state=0)
    print("Decision Tree: Fit")
    tree.fit(X_train, Y_train)
    
    print("Decision Tree: Predict")
    y_pred = tree.predict(X_test)
    print('Accuracy of Decision Tree classifier on train set: {:.2f}'.format(tree.score(X_train, Y_train)))
    print('Accuracy of Decision Tree classifier on test set: {:.2f}'.format(tree.score(X_test, Y_test)))
    
    cnf_matrix_dt = confusion_matrix(Y_test, y_pred)
    print("Decision Tree: Confusion Matrix")
    print(cnf_matrix_dt)
    print("Decision Tree: Classification Report")
    print(classification_report(Y_test,y_pred))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    print("Decision Tree: End")

### K-N-N

In [ ]:
if enable_knn:
    print("KNN: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
    print("KNN: Fit")
    knn.fit(X_train, Y_train)
    
    print("KNN: Predict")
    y_pred = knn.predict(X_test)
    print('Accuracy of KNN classifier on train set: {:.2f}'.format(knn.score(X_train, Y_train)))
    print('Accuracy of KNN classifier on test set: {:.2f}'.format(knn.score(X_test, Y_test)))
    
    print("KNN: Confusion Matrix")
    cnf_matrix_knn = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_knn)

    print("KNN: Classification Report")
    print(classification_report(Y_test,y_pred))

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))

    print("KNN: End")

### SVM

In [ ]:
if enable_svm:
    print("SVM: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    #svm = SVC(C=1, random_state=0, kernel='sigmoid', verbose=True)
    #svm = SVC(C=1, random_state=0, kernel='linear', verbose=True, cache_size=200)
    #svm = SVC(C=svm_c, gamma=svm_gamma, verbose = verbose_level)
    svm = SVC(C=1, verbose = verbose_level)
    print("SVM: Fit")
    svm.fit(X_train, Y_train)
    
    print("SVM: Predict")
    y_pred = svm.predict(X_test)
    
    print('Accuracy of SVM classifier on train set: {:.2f}'.format(svm.score(X_train, Y_train)))
    print('Accuracy of SVM classifier on test set: {:.2f}'.format(svm.score(X_test, Y_test)))
    
    print("SVM: Confusion Matrix")
    cnf_matrix_svm = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_svm)
    
    print("SVM: Classfication Report")
    print(classification_report(Y_test,y_pred))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("SVM: End")

## ANN - Multilayer Perceptron

In [ ]:
if enable_mlp:
    print("Multilayer Preceptron: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    
    #mlpc = MLPClassifier(alpha=1)
    mlpc = MLPClassifier(hidden_layer_sizes=(12, 12, 12), max_iter=100, verbose=verbose_level)
    
    #mlp = multilayer_perceptron(n_hidden =2, activation='logistic', algorithm='sgd', random_state=3)
    print("Multilayer Preceptron: fit")
    mlpc.fit(X_train, Y_train)
    
    print("Multilayer Preceptron: Predict")
    y_pred = mlpc.predict(X_test)

    print('Accuracy of Multilayer Perceptron classifier on train set: {:.2f}'.format(mlpc.score(X_train, Y_train)))
    print('Accuracy of Multilayer Perceptron classifier on test set: {:.2f}'.format(mlpc.score(X_test, Y_test)))
    
    print("Multilayer Preceptron: Confusion Matrix")
    cnf_matrix_mlp = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_mlp)
    
    print("Multilayer Preceptron: Classificiation Report")
    print(classification_report(Y_test,y_pred))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Multilayer Preceptron: End")

### Random forest

In [ ]:
if enable_rf:
    print("Ensemble (Bagging): Random Forest: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    forest = RandomForestClassifier(criterion='entropy', n_estimators=50, random_state=0, n_jobs=2, verbose=verbose_level)
    print("Ensemble (Bagging): Random Forest: Fit")
    forest.fit(X_train, Y_train)
    
    print("Ensemble (Bagging): Random Forest: Predict")
    y_pred = forest.predict(X_test)
    
    print('Accuracy of RandomForest classifier on train set: {:.2f}'.format(forest.score(X_train, Y_train)))
    print('Accuracy of RandomForest classifier on test set: {:.2f}'.format(forest.score(X_test, Y_test)))
    
    print("Ensemble (Bagging): Random Forest: Confusion Matrix")
    cnf_matrix_rf = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_rf)
    
    print("Ensemble (Bagging): Random Forest: Classification Report")
    print(classification_report(Y_test,y_pred))

    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    
    print("Ensemble (Bagging): Random Forest: End")

In [ ]:

#check sigmoid and rbf
#from sklearn.ensemble import BaggingClassifier
#from sklearn.svm import SVC
#clf = BaggingClassifier(SVC(C=1.0,
#        cache_size=200,
#        class_weight=None,
#        coef0=0.0,
#        decision_function_shape=None,
#        degree=3,
#        gamma='auto',
#        kernel='linear',
#        max_iter=-1,
#        probability=False,
#        random_state=None,
#        shrinking=True,
#        tol=0.001,
#        verbose=False,
#        ))

In [ ]:
#clf.fit(X_train, Y_train)

In [ ]:
t_end =  time.time()
print(time.asctime( time.localtime(t_end) ))

In [ ]:
#sys.stdout.close()
%logstop